# CSC 522 Project 
_by Team 21: Raphael Phillips (rjphill4), Sogolsadat Mansouri (smansou2), Rithik Jain (rjain25), and Neeloy Gomes (ntgomes)_

This project will involve developing a machine learning model using supervised learning to predict future exchange rates for a specific currency pair, based on historical exchange rate data and other relevant features.


In [19]:
%run data_cleaner.py

uscinrx_df = pd.read_csv('../data/USDINRX.csv')
cleaned_uscinrx_df = remove_rows_with_null_values(uscinrx_df)
print('Cleaned ' + str(len(uscinrx_df.index) - len(cleaned_uscinrx_df.index)) + ' rows from data/USDINRX.csv')

forex_df = pd.read_csv('../data/forex.csv')
cleaned_forex_df = remove_rows_with_null_values(forex_df)
print('Cleaned ' + str(len(forex_df.index) - len(cleaned_forex_df.index)) + ' rows from data/forex.csv')

Cleaned 52 rows from data/USDINRX.csv
Cleaned 0 rows from data/forex.csv


In [3]:
#%run exploratory_analysis.py

#do_analysis()

In [29]:
%run pre_processing2.py

pre_p_forex_df = pre_p_forex(cleaned_forex_df)

pre_p_forex_df

,date,open,high,low,close
1299751,12387,45.709000,45.728001,45.615002,45.709999
1299752,12388,45.709000,45.719002,45.560001,45.629002
1299753,12389,45.632000,45.655998,45.474998,45.549999
1299754,12390,45.548000,45.612999,45.519001,45.548000
1299755,12391,45.549999,45.566002,45.449001,45.449001
...,...,...,...,...,...
1304325,18863,74.105904,74.219498,74.055000,74.105904
1304326,18864,74.144897,74.301804,74.094299,74.144798
1304327,18865,74.101799,74.267502,74.068001,74.099998
1304328,18866,74.156197,74.330002,73.438004,74.155998


In [30]:
pre_p_usdinrx_df = pre_p_usdinrx(cleaned_uscinrx_df)

pre_p_usdinrx_df


,Date,Open,High,Low,Close
0,12387,45.709000,45.728001,45.615002,45.709999
1,12388,45.709000,45.719002,45.560001,45.629002
2,12389,45.632000,45.655998,45.474998,45.549999
3,12390,45.548000,45.612999,45.519001,45.548000
4,12391,45.549999,45.566002,45.449001,45.449001
...,...,...,...,...,...
4645,18890,73.696198,73.899597,73.607002,73.696098
4646,18891,73.659302,73.790100,73.549004,73.658997
4647,18892,73.795097,73.941498,73.650002,73.792702
4648,18893,73.875000,73.925797,73.588997,73.874901


In [33]:
final_df = merge(pre_p_forex_df, pre_p_usdinrx_df)

final_df

,Date,Open,High,Low,Close
0,12387,45.709000,45.728001,45.615002,45.709999
1,12388,45.709000,45.719002,45.560001,45.629002
2,12389,45.632000,45.655998,45.474998,45.549999
3,12390,45.548000,45.612999,45.519001,45.548000
4,12391,45.549999,45.566002,45.449001,45.449001
...,...,...,...,...,...
4645,18890,73.696198,73.899597,73.607002,73.696098
4646,18891,73.659302,73.790100,73.549004,73.658997
4647,18892,73.795097,73.941498,73.650002,73.792702
4648,18893,73.875000,73.925797,73.588997,73.874901
